# Introduction 

## Short description of the problem

For any number of arbitrary points, provided in (X,Y,Z) or (R,Z,phi) or (R,Z) coordinates, we want to efficiently test whether they lie inside the volume defined by a toroidal vacuum vessel.
This problem can be fully vectorized, it simply boils down to finding the most efficient routine for testing whether points lie inside or outside of a given polygon 

## Preliminary code for coordinates shifting 

In [49]:
def CoordShift_numpy(Pts, In='(X,Y,Z)', Out='(R,Z)', CrossRef=0.):
    """ Check the shape of an array of points coordinates and/or converts from 2D to 3D, 3D to 2D, cylindrical to cartesian... (CrossRef is an angle (Tor) or a distance (X for Lin))"""
    Lstr = ['(x,y,z)','(r,phi,z)','(r,z,phi)','(r,z)','(x,y)','(y,z)']
    assert all([type(ff) is str and ff.lower() in Lstr for ff in [In,Out]]), "Arg In and Out (coordinate format) must be in "+str(Lstr)+" !"
    assert type(Pts) is np.ndarray and Pts.ndim in [1,2] and Pts.shape[0] in (2,3), "Points must be a 1D or 2D np.ndarray of 2 or 3 coordinates !"
    assert type(CrossRef) in [int,float,np.int64,np.float64], "Arg CrossRef must be a float !"
    
    # Pre-format inputs
    In, Out = In.lower(), Out.lower()
    if Out==In:
        return Pts
    ndim = Pts.ndim
    if ndim==1:
        Pts = np.copy(Pts.reshape((Pts.shape[0],1)))

    if In=='(r,z,phi)':
        Pts = np.vstack((Pts[0,:],Pts[2,:],Pts[1,:]))
        In = '(r,phi,z)'
    
    mod = False
    if Out=='(r,z,phi)':
        Out = '(r,phi,z)'
        mod = True
    
    # Preliminary checks
    NP = Pts.shape[1]
    if Pts.shape[0]==2:
        assert In in ['(r,z)','(x,y)','(y,z)'], "Inconsistent input !"

    # Compute
    if Out=='(r,phi,z)':
        assert In in ['(r,z)','(x,y,z)','(x,y)'], "Not compatible !"
        if In=='(r,z)':
            pts = np.array([Pts[0,:], CrossRef*np.ones((NP,),dtype=float), Pts[1,:]])
        elif In=='(x,y,z)':
            R = np.hypot(Pts[0,:],Pts[1,:])
            Theta = np.arctan2(Pts[1,:],Pts[0,:])
            pts = np.array([R,Theta,Pts[2,:]])
        elif In=='(x,y)':
            R = np.hypot(Pts[0,:],Pts[1,:])
            Theta = np.arctan2(Pts[1,:],Pts[0,:])
            pts = np.array([R,Theta,CrossRef*np.ones((NP,),dtype=float)])
    elif Out=='(x,y,z)':
        assert In in ['(r,z)','(x,y)','(y,z)','(r,phi,z)'], "Not compatible !"
        if In=='(r,z)':
            pts = np.array([Pts[0,:]*np.cos(CrossRef), Pts[0,:]*np.sin(CrossRef), Pts[1,:]])
        elif In=='(x,y)':
            pts = np.array([Pts[0,:],Pts[1,:],CrossRef*np.ones((NP,),dtype=float)])
        elif In=='(y,z)':
            pts = np.array([CrossRef*np.ones((NP,),dtype=float), Pts[0,:], Pts[1,:]])
        elif In=='(r,phi,z)':
            pts = np.array([Pts[0,:]*np.cos(Pts[1,:]), Pts[0,:]*np.sin(Pts[1,:]), Pts[2,:]])
    elif Out=='(y,z)':
        assert In in ['(x,y,z)','(x,y)'], "Not compatible !"
        if In=='(x,y,z)':
            pts = Pts[1:,:]
        elif In=='(x,y)':
            pts = np.array([Pts[1,:], CrossRef*np.ones((NP,),dtype=float)])
    elif Out=='(r,z)':
        assert In in ['(r,phi,z)','(x,y,z)'], "Not compatible !"
        if In=='(r,phi,z)':
            pts = Pts[0::2,:]
        elif In=='(x,y,z)':
            pts = np.array([np.hypot(Pts[0,:],Pts[1,:]), Pts[2,:]])
        elif In=='(x,y)':
            pts = np.array([np.hypot(Pts[0,:],Pts[1,:]), CrossRef*np.ones((NP,),dtype=float)])
    elif Out=='(x,y)':
        assert In in ['(x,y,z)','(y,z)','(r,phi,z)'], "Not compatible !"
        if In=='(x,y,z)':
            pts = Pts[:-1,:]
        elif In=='(y,z)':
            pts = np.array([CrossRef*np.ones((NP,),dtype=float), Pts[0,:]])
        elif '(r,phi,z)':
            pts = np.array([Pts[0,:]*np.cos(Pts[1,:]), Pts[0,:]*np.sin(Pts[1,:])])
    
    # Format output
    if mod:
        pts = np.vstack((pts[0,:],pts[2,:],pts[1,:]))
    
    if ndim==1:
        pts = pts.flatten()
        
    return pts
        
        
        

In [4]:
%load_ext cython

In [50]:
%%cython
import numpy as np

def CoordShift(Pts, In='(X,Y,Z)', Out='(R,Z)', CrossRef=0.):
    """ Check the shape of an array of points coordinates and/or converts from 2D to 3D, 3D to 2D, cylindrical to cartesian... (CrossRef is an angle (Tor) or a distance (X for Lin))"""
    Lstr = ['(x,y,z)','(r,phi,z)','(r,z,phi)','(r,z)','(x,y)','(y,z)']
    assert all([type(ff) is str and ff.lower() in Lstr for ff in [In,Out]]), "Arg In and Out (coordinate format) must be in "+str(Lstr)+" !"
    assert type(Pts) is np.ndarray and Pts.ndim in [1,2] and Pts.shape[0] in (2,3), "Points must be a 1D or 2D np.ndarray of 2 or 3 coordinates !"
    assert type(CrossRef) in [int,float,np.int64,np.float64], "Arg CrossRef must be a float !"
    
    # Pre-format inputs
    In, Out = In.lower(), Out.lower()
    if Out==In:
        return Pts
    
    ndim = Pts.ndim
    if ndim==1:
        Pts = np.copy(Pts.reshape((Pts.shape[0],1)))

    if In=='(r,z,phi)':
        Pts = np.vstack((Pts[0,:],Pts[2,:],Pts[1,:]))
        In = '(r,phi,z)'
    
    mod = False
    if Out=='(r,z,phi)':
        Out = '(r,phi,z)'
        mod = True
    
    # Preliminary checks
    NP = Pts.shape[1]
    if Pts.shape[0]==2:
        assert In in ['(r,z)','(x,y)','(y,z)'], "Inconsistent input !"

    # Compute
    if Out=='(r,phi,z)':
        assert In in ['(r,z)','(x,y,z)','(x,y)'], "Not compatible !"
        if In=='(r,z)':
            pts = np.array([Pts[0,:], CrossRef*np.ones((NP,),dtype=float), Pts[1,:]])
        elif In=='(x,y,z)':
            R = np.hypot(Pts[0,:],Pts[1,:])
            Theta = np.arctan2(Pts[1,:],Pts[0,:])
            pts = np.array([R,Theta,Pts[2,:]])
        elif In=='(x,y)':
            R = np.hypot(Pts[0,:],Pts[1,:])
            Theta = np.arctan2(Pts[1,:],Pts[0,:])
            pts = np.array([R,Theta,CrossRef*np.ones((NP,),dtype=float)])
    elif Out=='(x,y,z)':
        assert In in ['(r,z)','(x,y)','(y,z)','(r,phi,z)'], "Not compatible !"
        if In=='(r,z)':
            pts = np.array([Pts[0,:]*np.cos(CrossRef), Pts[0,:]*np.sin(CrossRef), Pts[1,:]])
        elif In=='(x,y)':
            pts = np.array([Pts[0,:],Pts[1,:],CrossRef*np.ones((NP,),dtype=float)])
        elif In=='(y,z)':
            pts = np.array([CrossRef*np.ones((NP,),dtype=float), Pts[0,:], Pts[1,:]])
        elif In=='(r,phi,z)':
            pts = np.array([Pts[0,:]*np.cos(Pts[1,:]), Pts[0,:]*np.sin(Pts[1,:]), Pts[2,:]])
    elif Out=='(y,z)':
        assert In in ['(x,y,z)','(x,y)'], "Not compatible !"
        if In=='(x,y,z)':
            pts = Pts[1:,:]
        elif In=='(x,y)':
            pts = np.array([Pts[1,:], CrossRef*np.ones((NP,),dtype=float)])
    elif Out=='(r,z)':
        assert In in ['(r,phi,z)','(x,y,z)'], "Not compatible !"
        if In=='(r,phi,z)':
            pts = Pts[0::2,:]
        elif In=='(x,y,z)':
            pts = np.array([np.hypot(Pts[0,:],Pts[1,:]), Pts[2,:]])
        elif In=='(x,y)':
            pts = np.array([np.hypot(Pts[0,:],Pts[1,:]), CrossRef*np.ones((NP,),dtype=float)])
    elif Out=='(x,y)':
        assert In in ['(x,y,z)','(y,z)','(r,phi,z)'], "Not compatible !"
        if In=='(x,y,z)':
            pts = Pts[:-1,:]
        elif In=='(y,z)':
            pts = np.array([CrossRef*np.ones((NP,),dtype=float), Pts[0,:]])
        elif '(r,phi,z)':
            pts = np.array([Pts[0,:]*np.cos(Pts[1,:]), Pts[0,:]*np.sin(Pts[1,:])])
    
    # Format output

    if mod:
        pts = np.vstack((pts[0,:],pts[2,:],pts[1,:]))

    if ndim==1:
        pts = pts.flatten()

    return pts
        

In [51]:
Pts = np.array([np.linspace(0,10.,1000), np.linspace(0,10,1000), np.linspace(0,10,1000)])
ptsn = CoordShift_numpy(Pts, In='(x,y,z)',Out='(r,phi,z)')
ptsc = CoordShift(Pts, In='(x,y,z)',Out='(r,z,phi)')

%timeit ptsn = CoordShift_numpy(Pts, In='(x,y,z)',Out='(r,z,phi)')
%timeit ptsc = CoordShift(Pts, In='(x,y,z)',Out='(r,z,phi)')

10000 loops, best of 3: 136 µs per loop
The slowest run took 5.29 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 113 µs per loop


## Code

In [ ]:
from matplotlib.path import Path

def _Ves_isInside_plg(Pts, VPoly, VLong=None, VType='Tor', In='(X,Y,Z)', Test=True):
    if Test:
        assert type(Pts) is np.ndarray and Pts.ndim in [1,2], "Arg Pts must be a 1D or 2D np.ndarray !"
        assert type(VPoly) is np.ndarray and VPoly.ndim==2 and VPoly.shape[0]==2, "Arg VPoly must be a (2,N) np.ndarray !"
        assert VLong is None or (hasattr(VLong,'__iter__') and len(VLong)==2), "Arg VLong must be a len()==2 iterable !"
        assert type(VType) is str and VType.lower() in ['tor','lin'], "Arg VType must be a str in ['Tor','Lin'] !"
        
    path = Path(Poly.T)
    if Vtype.lower()=='tor':
        Pts = CoordShift(Pts, In=In, Out='(R,Z)')
        ind = Path.contains_points(Pts.T, transform=None, radius=0.0)
    else:
        Pts = CoordShift(Pts, In=In, Out='(X,Y)')
        ind = Path.contains_points(Pts.T, transform=None, radius=0.0)
        if In.lower()=='(x,y,z)':
            ind = ind & (Pts[2,:]>=VLong[0]) & (Pts[2,:]<=Vlong[1])
    return ind
    
